<a href="https://colab.research.google.com/github/iampragyagupta/Deep-Learning-for-Computer-Vision/blob/main/lbp_with_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.neighbors import NearestNeighbors

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
images = []
labels = []

for i in train_images:
  images.append(i)

for i in train_labels:
  labels.append(i)

In [ ]:
#lbp function

def lbp_r1(img_vector):
  dim_y,dim_x= img_vector.shape
  image = np.zeros((dim_y+2, dim_x+2), dtype=np.uint32)  #padded 0s
  for i in range(1, dim_y + 1):
        for j in range(1, dim_x + 1):
          image[i,j] = img_vector[i-1,j-1]

  lbp_image = np.zeros((dim_y, dim_x), dtype=np.uint32)
  for i in range(1, dim_y + 1):

        for j in range(1, dim_x + 1):
            binary_val = 0
            center = image[i, j]
            neighbors = [
                image[i, j+1], image[i-1, j+1], image[i-1, j], image[i-1, j-1],  #right, right-top, top, left-top
                image[i, j-1], #left
                image[i+1, j-1], image[i+1, j], image[i+1, j+1],  #left down, down, right down

            ]
            count=0
            for neighbor in neighbors:
              if(neighbor>=center):
                binary_val= binary_val+ 2**(count)
              count=count+1
            lbp_image[i-1, j-1] = binary_val

  return lbp_image

In [ ]:
# LBP with radius 2.

#lbp function r2
def lbp_r2(img_vector):
  dim_y,dim_x= img_vector.shape
  image = np.zeros((dim_y+4, dim_x+4), dtype=np.uint32)  #padded 0s
  for i in range(2, dim_y + 2):
        for j in range(2, dim_x + 2):
          image[i,j] = img_vector[i-2,j-2]

  lbp_image = np.zeros((dim_y, dim_x), dtype=np.uint32)
  for i in range(2, dim_y + 2):

        for j in range(2, dim_x + 2):
            binary_val = 0
            center = image[i, j]
            neighbors = [
                image[i, j+2], image[i-1, j+2], image[i-2, j+2], image[i-2, j+1], image[i-2, j], image[i-2, j-1], image[i-2, j-2], #right, right-top, top, left-top
                image[i-1, j-2], image[i, j-2], image[i+1, j-2], image[i+2, j-2], #left
                image[i+2, j-1], image[i+2, j], image[i+2, j+1], image[i+2, j+2], image[i+1, j+2], #left down, down, right down

            ]
            count=0
            # print(center)
            # print(neighbors)
            for neighbor in neighbors:
              if(neighbor>=center):
                binary_val= binary_val+ 2**count
              count=count+1
            lbp_image[i-2, j-2] = binary_val

  return lbp_image

In [ ]:
#RUN ONLY IF YOU WANT TO USE LBP APPROACH OR THE MODIFIED (RADIUS = 2) LBP APPROACH.
#TO SEE THE PROPOSED GRID APPROACH, GO TO THE NEXT CODE BLOCK

#apply lbp on the train set and produce histogram
lbp_images = np.array([lbp_r1(img) for img in images]) # can be changed to lbp_r2, based on the need

# Function to Compute Histogram of LBP Patterns
def compute_lbp_histogram(lbp_image, num_bins=256):
    hist, _ = np.histogram(lbp_image.flatten(), bins=num_bins, range=(0, 256))
    hist = hist.astype("float") / hist.sum()  # Normalizing the histogram
    return hist

# Compute histograms for all images
histograms = np.array([compute_lbp_histogram(img) for img in lbp_images])

In [ ]:
#pre grid LoG - will be used in the grid apprach
def log(img_vector):


  dim_y,dim_x= img_vector.shape
  image = np.zeros((dim_y+2, dim_x+2), dtype=np.uint8)  #padded 0s

  for i in range(1, dim_y + 1):
        for j in range(1, dim_x + 1):
          image[i,j] = img_vector[i-1,j-1]

  lbp_image = np.zeros((dim_y, dim_x), dtype=np.int32)

  for i in range(1, dim_y + 1):

        for j in range(1, dim_x + 1):
            binary_val = 0
            center = image[i, j]
            neighbors = [
                image[i, j+1], image[i-1, j+1], image[i-1, j], image[i-1, j-1],  #right, right-top, top, left-top
                image[i, j-1], #left
                image[i+1, j-1], image[i+1, j], image[i+1, j+1],  #left down, down, right down

            ]

            sum_of_neighbors = 0
            for neighbor in neighbors:
              sum_of_neighbors += neighbor

            new_val = 8*center - sum_of_neighbors
            if(new_val < 0):
              new_val = 0
            elif(new_val > 255):
              new_val = 255

            lbp_image[i-1, j-1] = new_val


  for i in range(1, dim_y + 1):

        for j in range(1, dim_x + 1):
            binary_val = 0
            center = image[i, j]
            neighbors = [
                image[i, j+1], image[i-1, j+1], image[i-1, j], image[i-1, j-1],  #right, right-top, top, left-top
                image[i, j-1], #left
                image[i+1, j-1], image[i+1, j], image[i+1, j+1],  #left down, down, right down

            ]

            sum_of_neighbors = 0
            for neighbor in neighbors:
              sum_of_neighbors += neighbor

            new_val = (center + sum_of_neighbors) / 9
            if(new_val < 0):
              new_val = 0
            elif(new_val > 255):
              new_val = 255

            lbp_image[i-1, j-1] = new_val

  return lbp_image

In [ ]:
# our approach to divide section into grids and run the LBP for each, and then concatenate the histograms
#grids

# Function to Compute LBP for Each Grid
def compute_lbp_histogram_grid(img, grid_size=(5, 5), num_bins=256):
    """
    Computes LBP histograms for each grid in the image.
    - img: Input image
    - grid_size: Tuple indicating the number of grids (rows, cols)
    - num_bins: Number of bins in the histogram
    Returns:
      Concatenated histograms from all grids
    """
    img = log(img)
    h, w = img.shape  # Get image dimensions
    grid_height, grid_width = divmod(h, grid_size[0])[0], divmod(w, grid_size[1])[0]
    histograms = []

    for row in range(grid_size[0]):  # Loop through grid rows
        row_start, row_end = row * grid_height, (row + 1) * grid_height

        for col in range(grid_size[1]):  # Loop through grid columns
            col_start, col_end = col * grid_width, (col + 1) * grid_width

            # Extract the current grid section from the image
            grid_section = img[row_start:row_end, col_start:col_end]

            center = np.median(grid_section)
            # Compute LBP features for this section
            lbp_features = lbp_r1(grid_section)

            # Generate histogram and normalize it
            histogram, _ = np.histogram(lbp_features.ravel(), bins=num_bins, range=(0, num_bins))
            histogram = histogram.astype(np.float32) / np.sum(histogram)  # Normalize

            # Store the histogram
            histograms.append(histogram)


    return np.concatenate(histograms)  # Return concatenated histogram

# Compute LBP histograms for all images with grids
lbp_histograms_grid = np.array([compute_lbp_histogram_grid(img, grid_size=(5, 5)) for img in images])
histograms=lbp_histograms_grid

In [ ]:
# once any one of the methods have been used and the histograms are made, we run a KNN algo

histograms = np.array(histograms)
knn = NearestNeighbors(n_neighbors=no_neighbors, metric='euclidean')
knn.fit(histograms)

In [ ]:
# calculated the closest neighbours for any feature vector

from sklearn.metrics import precision_score, recall_score, f1_score
def get_closest_images(image_index,train_labels):

    predictions = []
    true_labels = []

    # Retrieve the histogram of the input image
    input_hist = histograms[image_index].reshape(1, -1)

    # Find the indices of the 6 closest images (including the input image)
    distances, indices = knn.kneighbors(input_hist)

    # Exclude the input image from the list of neighbors
    indices = indices[0][indices[0] != image_index]
    indices = indices[:5]
    for i in indices:
      true_labels.append(train_labels[i])

    # UNHIDE TO SEE IMAGES OF CLOSEST NEIGHBOURS
    # PLS COMMENT OUT WHEN RUNNING THE FULL LOOP FOR PRECISION, RECALL, ETC.
    # Display the input image

    # plt.figure(figsize=(10, 6))
    # plt.subplot(2, 3, 1)
    # plt.imshow(images[image_index], cmap='gray')
    # plt.title(f'Input Image {image_index}')
    # plt.axis('off')

    # # Display the 5 closest images
    # for i, idx in enumerate(indices[:]):
    #     plt.subplot(2, 3, i + 2)
    #     plt.imshow(images[idx], cmap='gray')
    #     plt.title(f'Neighbor {i + 1}')
    #     plt.axis('off')

    # plt.tight_layout()
    # plt.show()


    return np.array(true_labels)


In [ ]:
no_neighbors= int(input("how many nearest neighbors would you like: ")) +1 # specify number of closest images you want to see

def create_numpy_list(length, value):
    return np.full(length, value)

def getScores(query_img):

  true_labels = get_closest_images(query_img,train_labels)
  pred_labels = create_numpy_list(n,labels[query_img])

  precision = precision_score(true_labels, pred_labels, average='macro', zero_division=0)
  recall = recall_score(true_labels, pred_labels,average='macro', zero_division=0)
  f1 = f1_score(true_labels, pred_labels,average='macro', zero_division=0)

  return precision,recall,f1


In [ ]:
pr = []
re = []
fs = []

for i in range(100): # range is the number of samples that the code will check the nearest neighbours for, and record the precision, recall etc.
  #print(i,p,r,f)
  p,r,f = getScores(i)

  pr.append(p)
  re.append(r)
  fs.append(f)

precision = sum(pr)/len(pr)
recall = sum(re)/len(re)
fscore = sum(fs)/len(fs)

print(precision) # average values of precision, recall etc are printed
print(recall)
print(fscore)